In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, t) for w, t in zip(s['words'].values.tolist(),
                                                       s['labels'].values.tolist())]
        self.grouped = self.df.groupby("sentence_id").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

#  Preprocess- Marathi



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import sklearn

In [ ]:
# path='/content/drive/MyDrive/MarathiNER/NotCollapsed/marathi.train'
# path1='/content/drive/MyDrive/MarathiNER/NotCollapsed/marathi.test'
# path2 = '/content/drive/MyDrive/MarathiNER/NotCollapsed/marathi.tune'

# onkar
path='/content/drive/MyDrive/L3Cube_Datasets/NotCollapsed/marathi.train'
path1='/content/drive/MyDrive/L3Cube_Datasets/NotCollapsed/marathi.test'
path2 = '/content/drive/MyDrive/L3Cube_Datasets/NotCollapsed/marathi.tune'


#parth
# path='/content/drive/MyDrive/Marathi_dataset/NotCollapsed/marathi.train'
# path1='/content/drive/MyDrive/Marathi_dataset/NotCollapsed/marathi.test'
# path2 = '/content/drive/MyDrive/Marathi_dataset/NotCollapsed/marathi.tune'



In [ ]:
with open(path,'r') as f:
  content=f.readlines()
  
with open(path1,'r') as f:
  content1=f.readlines()

with open(path2,'r') as f:
  content2=f.readlines()

In [ ]:
content= [ x.replace('\t', ' ') for x in content]
content= [ x.replace('\n', ' ') for x in content]

content1= [ x.replace('\t', ' ') for x in content1]
content1= [ x.replace('\n', ' ') for x in content1]

content2= [ x.replace('\t', ' ') for x in content2]
content2= [ x.replace('\n', ' ') for x in content2]

In [ ]:
pos=[]

for i in content:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    pos.append(t[1])
  else:
    pos.append(" ")

tag=[]
for i in content:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    tag.append(t[2])
  else:
    tag.append(" ")

word=[]
for i in content:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    word.append(t[0])
  else:
    word.append(" ")

# --------------------------------------------------------------

pos1=[]

for i in content1:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    pos1.append(t[1])
  else:
    pos1.append(" ")

tag1=[]
for i in content1:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    tag1.append(t[2])
  else:
    tag1.append(" ")

word1=[]
for i in content1:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    word1.append(t[0])
  else:
    word1.append(" ")

# --------------------------------------------------------

pos2=[]

for i in content2:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    pos2.append(t[1])
  else:
    pos2.append(" ")

tag2=[]
for i in content2:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    tag2.append(t[2])
  else:
    tag2.append(" ")

word2=[]
for i in content2:
  sample=''
  sample=i
  # print(sample)
  if(sample!=' '):
    t=sample.split()
    word2.append(t[0])
  else:
    word2.append(" ")

In [ ]:
train = pd.DataFrame(
    {'words': word,
     'POS': pos,
     'labels': tag
    })

test = pd.DataFrame(
    {'words': word1,
     'POS': pos1,
     'labels': tag1
    })

tune = pd.DataFrame(
    {'words': word2,
     'POS': pos2,
     'labels': tag2
    })

In [ ]:
frames = [train, test, tune]
df = pd.concat(frames)

In [ ]:
def preprocess(final):
  final1= final[final['words'] != ' ']
  final = final1
  final=  final[final['words']!=',']
  final=final.reset_index()
  final=final.drop('index', axis=1)
  final['sentence_id']=np.nan
  # final = final.replace(to_replace ="B-LOCATION",value ="BLOCATION")
  # final = final.replace(to_replace ="I-LOCATION",value ="ILOCATION")
  # final = final.replace(to_replace ="B-PERSON",value ="BPERSON")
  # final = final.replace(to_replace ="I-PERSON",value ="IPERSON")
  # final = final.replace(to_replace ="B-ORGANIZATION",value ="BORGANIZATION")
  # final = final.replace(to_replace ="I-ORGANIZATION",value ="IORGANIZATION")
  s=1
  final['sentence_id'][0]=s
  s_count=2
  s=[]
  for i in range (0,len(final)):
    if final['words'][i]=='.' or final['words'][i]=='?' or final['words'][i]=='!':
      final['sentence_id'][i+1]=s_count
      s.append(s_count)
      s_count=s_count+1
  final = final.fillna(method = 'ffill')
  o_noob=[]
  for i in range(0, len(final)):
    o_noob.append(int(final['sentence_id'][i]))
  final['sentence_id']=pd.DataFrame(o_noob)
  df_jud = final
  remove_jud = final.labels.unique()
  remove_jud = remove_jud.tolist()
  print(remove_jud)
  elements = ['B-LOCATION','I-LOCATION', 'B-PERSON','I-PERSON', 'B-ORGANIZATION','I-ORGANIZATION']
  # elements = ['B-LOCATION','I-LOCATION', 'B-ORGANIZATION','I-ORGANIZATION']
  for ele in elements:
    remove_jud.remove(ele)
  for i in range(0, len(df_jud)):
    if df_jud['labels'][i] in remove_jud:
      df_jud['labels'][i]='O'
  return final

In [ ]:
def to_non_iob(final):
  final = final.replace(to_replace ="B-LOCATION",value ="LOCATION")
  final = final.replace(to_replace ="I-LOCATION",value ="LOCATION")
  final = final.replace(to_replace ="B-PERSON",value ="PERSON")
  final = final.replace(to_replace ="I-PERSON",value ="PERSON")
  final = final.replace(to_replace ="B-ORGANIZATION",value ="ORGANIZATION")
  final = final.replace(to_replace ="I-ORGANIZATION",value ="ORGANIZATION")

  return final

In [ ]:
training_data_bert = preprocess(train)
testing_data_bert = preprocess(test)
tuning_data_bert = preprocess(tune)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['O', 'B-LOCATION', 'I-LOCATION', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-PERSON', 'I-PERSON']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['O', 'B-LOCATION', 'I-LOCATION', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-PERSON', 'I-PERSON']
['B-LOCATION', 'O', 'I-LOCATION', 'B-PERSON', 'I-PERSON', 'B-ORGANIZATION', 'I-ORGANIZATION']


In [ ]:
training_data_bert = to_non_iob(training_data_bert)
testing_data_bert = to_non_iob(testing_data_bert)
tuning_data_bert = to_non_iob(tuning_data_bert)

In [ ]:
# #Displaying one full sentence
# getter = sentence(testing_data_bert)
# sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
# sentences[0]

In [ ]:
training_data_bert = training_data_bert.drop(['POS'], axis=1)
testing_data_bert = testing_data_bert.drop(['POS'], axis=1)
tuning_data_bert = tuning_data_bert.drop(['POS'], axis=1)

In [ ]:
cols = ['sentence_id', 'words', 'labels']

In [ ]:
training_data_bert = training_data_bert[cols]
testing_data_bert = testing_data_bert[cols]
tuning_data_bert = tuning_data_bert[cols]

In [ ]:
training_data_bert.labels.value_counts()

O               60830
LOCATION         5226
PERSON           1546
ORGANIZATION      173
Name: labels, dtype: int64

In [ ]:
training_data_bert.tail()

,sentence_id,words,labels
67770,3588,महल,LOCATION
67771,3588,बघणे,O
67772,3588,चुकवू,O
67773,3588,नका,O
67774,3588,.,O


# **Transformers- Marathi**

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 1.8 MB 37.6 MB/s 
     |████████████████████████████████| 10.1 MB 33.6 MB/s 
     |████████████████████████████████| 4.2 MB 34.9 MB/s 
     |████████████████████████████████| 342 kB 45.8 MB/s 
     |████████████████████████████████| 6.6 MB 16.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 1.2 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 145 kB 44.9 MB/s 
     |████████████████████████████████| 181 kB 43.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 136 kB 47.8 MB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
     |████████████████████████████████| 212 kB 49.1 MB/s 
     |████████████████████████████████| 127 kB 50.1 MB/s 
     |██████████████

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
# new_tags=['O','B-LOCATION', 'I-LOCATION', 'B-PERSON', 'I-PERSON', 'I-ORGANIZATION', 'B-ORGANIZATION']

new_tags=['O','LOCATION', 'PERSON', 'ORGANIZATION']

In [ ]:
model_args = NERArgs()
# model_args.num_train_epochs = 3
model_args.evaluate_during_training = True
model_args.train_batch_size = 16

### **1) Multicase BERT (mBert)**

In [ ]:
# model_args.train_custom_parameters_only = True
# model_args.custom_parameter_groups = [
#     {
#         "params": [
#               "classifier.weight",
#  "classifier.bias"]
        
#     }

# ]

# # "bert.embeddings.word_embeddings.weight",
# #                 "bert.embeddings.position_embeddings.weight",
# #                 "bert.embeddings.token_type_embeddings.weight",
# #                 "bert.embeddings.LayerNorm.weight",
# #               "bert.embeddings.LayerNorm.bias",

In [ ]:
custom_labels = new_tags

mbert = NERModel(
    "bert", "bert-base-multilingual-cased", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
for i in range(0,10):
  !rm -rf /content/outputs
  mbert.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = mbert.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


KeyboardInterrupt: ignored

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

[83, 113, 164, 297, 434, 452, 567, 610, 716, 766, 896, 904, 995, 1075, 1094, 1158, 1190, 1197, 1217, 1219, 1238, 1258, 1266, 1395, 1418, 1445, 1478, 1484, 1489, 1507, 1522, 1530]
32


In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9242473502828532

### **2) Indic BERT**

In [ ]:
custom_labels = new_tags

indicbert = NERModel(
    "albert", "ai4bharat/indic-bert", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

In [ ]:
for i in range(0,10):
  !rm -rf /content/outputs
  indicbert.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = indicbert.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

{'eval_loss': 0.29906198906549736, 'precision': 0.6141253507951356, 'recall': 0.5615911035072711, 'f1_score': 0.5866845397676497}


In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

[904, 1484]
2


In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)

0.9279770623617041



### **3)Roberta- marathi**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

roberta_marathi = NERModel(
     "roberta", "flax-community/roberta-base-mr", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at flax-community/roberta-base-mr were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at flax-community/roberta-base-mr and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task 

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

In [ ]:
for i in range(0, 10):
  !rm -rf /content/outputs
  roberta_marathi.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = roberta_marathi.eval_model(testing_data_bert)
  print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/192 [00:00<?, ?it/s]

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **4)Xlm-roberta**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "xlmroberta", "xlm-roberta-base", labels=custom_labels, args =model_args
)

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **5)MahaBERT**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "bert", "l3cube-pune/marathi-bert", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/828 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Some weights of the model checkpoint at l3cube-pune/marathi-bert were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert and are

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/304 [00:00<?, ?B/s]

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **6)MahaRoBERTa**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "xlmroberta", "l3cube-pune/marathi-roberta", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at l3cube-pune/marathi-roberta were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at l3cube-pune/marathi-roberta and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **7)MahaAlBERT**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "albert", "l3cube-pune/marathi-albert-v2", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/838 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/126M [00:00<?, ?B/s]

Some weights of the model checkpoint at l3cube-pune/marathi-albert-v2 were not used when initializing AlbertForTokenClassification: ['predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at l3cube-pune/marathi-albert-v2 and are newly initialized: ['classifier.weight', 'classif

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **8)RoBERTa Hindi**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "roberta", "flax-community/roberta-hindi", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/692 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at flax-community/roberta-hindi were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at flax-community/roberta-hindi and are newly initialized: ['classifier.weight', 'classifier.bias']
You should pr

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)



### **9)indic-transformers-hi-roberta**

In [ ]:
custom_labels = new_tags
# model = NERModel(
#     "roberta", "flax-community/roberta-base-mr"
# )

xlmroberta = NERModel(
     "roberta", "neuralspace-reverie/indic-transformers-hi-roberta", labels=custom_labels, args =model_args
)

Downloading:   0%|          | 0.00/537 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-hi-roberta were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at neuralspace-reverie/indic-transformers-hi-roberta and are newly initialized: ['classifier

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/959k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
for i in range(10):
  !rm -rf /content/outputs
  xlmroberta.train_model(training_data_bert, eval_data=tuning_data_bert)
  result, model_outputs, preds_list = xlmroberta.eval_model(testing_data_bert)
  print(result)

In [ ]:
test_data = testing_data_bert

In [ ]:
getter = sentence(test_data)
test_list = [" ".join([s[1] for s in sent]) for sent in getter.sentences]

In [ ]:
for i in range(0, len(test_list)):
  test_list[i] = test_list[i].split()

In [ ]:
wrong_indices = []

for i in range(0, len(preds_list)):
  if(len(preds_list[i]) != len(test_list[i])):
    wrong_indices.append(i)

print(wrong_indices)
print(len(wrong_indices))

In [ ]:
test = []
pred = []

for i in range(0, len(preds_list)):
  if i not in wrong_indices:
    test.append(test_list[i])
    pred.append(preds_list[i])

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
test = flatten(test)
pred = flatten(pred)

In [ ]:
accuracy_score(test, pred)